<a href="https://colab.research.google.com/github/viniciusramaral/deeplearning_prever_valor_aluguel/blob/main/CP03_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsão do valor de aluguel

In [ ]:
#Entendo que o Colab contenha as bibliotecas instaladas.
#Porém deixo o código abaixo para depois fazer a instalação em meu equipamento
#!pip install tensorflow pandas scikit-learn


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
#Carregadndo os dados dentro do Pandas
#Foi necessário informar ao pandas qual o caractere de separação de colunas
data = pd.read_csv('aluguel.csv', sep=";")

In [ ]:
#Exibindo alguns dados da tabela
data.head()

,cidade,area,quartos,banheiros,vagas_garagem,andar,aceita_animal,mobiliaria,valor_condominio,valor_aluguel,valor_iptu,valor_seguro,valor_total
0,São Paulo,70,2,1,1,7,1,1,2065,3300,211,42,5618
1,São Paulo,320,4,4,0,20,1,0,1200,4960,1750,63,7973
2,Porto Alegre,80,1,1,1,6,1,0,1000,2800,0,41,3841
3,Porto Alegre,51,2,1,0,2,1,0,270,1112,22,17,1421
4,São Paulo,25,1,1,0,1,0,0,0,800,25,11,836


In [ ]:
#Abaixo verifico quais os tipos de dados nas colunas
#Neste caso a única preocupação é com a cidade, na qual carrega strings
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10685 entries, 0 to 10684
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   cidade            10685 non-null  object
 1   area              10685 non-null  int64 
 2   quartos           10685 non-null  int64 
 3   banheiros         10685 non-null  int64 
 4   vagas_garagem     10685 non-null  int64 
 5   andar             10685 non-null  int64 
 6   aceita_animal     10685 non-null  int64 
 7   mobiliaria        10685 non-null  int64 
 8   valor_condominio  10685 non-null  int64 
 9   valor_aluguel     10685 non-null  int64 
 10  valor_iptu        10685 non-null  int64 
 11  valor_seguro      10685 non-null  int64 
 12  valor_total       10685 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 1.1+ MB


In [ ]:
# Separando as características e o alvo da previsão que teremos que fazer
X = data.drop('valor_total', axis=1)
y = data['valor_total']

In [ ]:
# Tratando a coluna de cidade que não é zero e um e sim um conjunto de strings
categorical_features = ['cidade']
X = pd.get_dummies(X, columns=categorical_features, drop_first=True)

In [ ]:
#Dividindo os dados entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Normalização dos dados
#Aqui os dados foram normalizados para que todos eles tenha magnitude similar.
#Talvez aqui, para esse propósito, não seja necessário.
#Porém como pretendo usar esse mesmo script para outros tipos de previsão, como o de reconhecimento facial, estou testando esta questão.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Definindo o modelo
# Aqui aplica-se a função de ativação. Ela é responsável pelo aprendizado de um neurônio.
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

In [ ]:
# Compilando o modelo
# Aqui otimizamos o código para que ele tenha melhor taxa de aprendizado se auto-adaptadando a cada resposta
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Resumo do modelo
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                1024      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3137 (12.25 KB)
Trainable params: 3137 (12.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Treinamento do modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)



Epoch 1/100
214/214 [==============================] - 1s 4ms/step - loss: 46422084.0000 - val_loss: 43144552.0000
Epoch 2/100
214/214 [==============================] - 1s 3ms/step - loss: 35177668.0000 - val_loss: 22557022.0000
Epoch 3/100
214/214 [==============================] - 1s 3ms/step - loss: 13098922.0000 - val_loss: 6874146.5000
Epoch 4/100
214/214 [==============================] - 1s 4ms/step - loss: 5101579.0000 - val_loss: 5022041.0000
Epoch 5/100
214/214 [==============================] - 1s 4ms/step - loss: 3831850.5000 - val_loss: 4120339.7500
Epoch 6/100
214/214 [==============================] - 1s 3ms/step - loss: 3000809.7500 - val_loss: 3405469.7500
Epoch 7/100
214/214 [==============================] - 0s 2ms/step - loss: 2311378.0000 - val_loss: 2781837.2500
Epoch 8/100
214/214 [==============================] - 1s 3ms/step - loss: 1738519.2500 - val_loss: 2245613.2500
Epoch 9/100
214/214 [==============================] - 0s 2ms/step - loss: 1270593.2500 - v

In [ ]:
# Avaliação do modelo
# O meu primeiro teste eu tive 146 de EQM
# Fiz um teste tirando o andar dos cálculos e meu modelo performou menos aumentando o EQM para 229
#Ao remover o valor do seguro, meu EQM aumentou, porém quase que insignificantemente. Foi para 147
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error: {loss}')

67/67 [==============================] - 0s 3ms/step - loss: 281.6324
Mean Squared Error: 281.6324157714844


In [ ]:
# Previsão
y_pred = model.predict(X_test)

# Exibindo algumas previsões
for i in range(5):
    print(f'Real: {y_test.iloc[i]}, Previsto: {y_pred[i][0]}')

#Todos os itens
#Real: 5301, Previsto: 5278.34423828125
#Real: 2081, Previsto: 2089.09326171875
#Real: 3800, Previsto: 3807.052490234375
#Real: 2730, Previsto: 2729.9404296875
#Real: 9377, Previsto: 9362.9345703125


#Ignorando aceita animal
#Real: 5301, Previsto: 5301.98681640625
#Real: 2081, Previsto: 2079.248046875
#Real: 3800, Previsto: 3800.467529296875
#Real: 2730, Previsto: 2722.161376953125
#Real: 9377, Previsto: 9375.3603515625

67/67 [==============================] - 0s 2ms/step
Real: 5301, Previsto: 5278.34423828125
Real: 2081, Previsto: 2089.09326171875
Real: 3800, Previsto: 3807.052490234375
Real: 2730, Previsto: 2729.9404296875
Real: 9377, Previsto: 9362.9345703125


Concluo que:


1.   Ainda tenho dificuldade em avaliar se ao passar das épocas o código está performando melhor ou não.
2.   Os resultados foram praticamente iguais quando se trata de ter animal ou não no imóve.
2.   A previsão do aluguel foi excelente para as amostragens testadas.

